# First Jupyter Notebook with the Bluesky Framework : 2021-03 Training Edition

TODO:

See also [Guide: First Steps with Bluesky](https://github.com/BCDA-APS/use_bluesky/blob/main/first_steps_guide.md), a quick-reference guide for those learning how to use the Bluesky Framework for data acquisition.  This guide provides the basic commands you will need when using Bluesky from an IPython console or a Jupyter notebook.

## Start the `instrument` package

In [ ]:
# %matplotlib inline
from instrument.collection import *

## Where _is_ everybody?

On the command line, there is an [IPython magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html#built-in-magic-commands) command from the [bluesky](https://blueskyproject.io/bluesky/) package to print the current value of [labeled](https://blueskyproject.io/bluesky/magics.html?highlight=label) items: [`%wa`]()

In [ ]:
%wa

## **Who** is everybody?

Might be a good idea to know now what ophyd symbols are available.  The [apstools](https://apstools.readthedocs.io/en/latest/) package provides a [`listobjects()`](https://apstools.readthedocs.io/en/latest/source/_utils.html?highlight=listobjects#apstools.utils.listobjects) command that prints a table of all the known objects (in the global namespace of the session).  The columns provide the ophyd name (the name you use to call this in Python), the name of the ophyd structure, the EPICS PV (if relevant), and any labels (as used in `%wa` above).

Notably, the table includes:

* a simulated EPICS area detector (`adsimdet`), look for the `area_detector` label
* a simulated 16-channel EPICS scaler (`scaler1`), look for the `scalers` label
* some of the scaler channels are named (`I0`, `diode`, ...), look for the `counter` label
* a _noisy_ detector for use with motor `m1` to simulate a diffraction peak
* 16 simulated EPICS motors (`m1` .. `m16`), look for the `motor` label
* some calculation support (`calcs` & `calcouts`)
* a simulated shutter (`shutter`)
* a simulated temperature controller (`temperature`)
* details about the general purpose IOC (`gp_stats`)

In [ ]:
listobjects()

## Let's work with the scaler

A scaler is a device that counts digital pulses from a pulse detector such
as a scintillation counter or from photodiodes or ionization chamber with
pulse chain electronics.  Scalers have
many channels, some of which might have no associated detector.  Our scaler
(`scaler1`) is a simulated device that records a random number of pulses in 
each channel.  We are only interested in the channels that have names provided
by users in the GUI screens.  In this screen for our scaler, only a few of the
channels are named:

![`scaler` GUI](resources/scaler.png "`scaler1` GUI")

Let's configure `scaler1` to report only the `diode` and `I0` channels (plus the count time channel which will *always* be included).  Keep in mind that the argument to this function is a Python list, so the channel names must be enclosed with `[]`.  The function does not return a result.  If something *is* printed, there is an error to be fixed.

NOTE:  To report _all_ named channels again, call the same function with argument in the parentheses.

In [ ]:
scaler1.select_channels(["diode", "I0"])

The _easiest_ way to count the `scaler` object is to use the [`%ct`](https://blueskyproject.io/bluesky/magics.html?highlight=label#taking-a-reading-using-ct-post-v1-3-0) bluesky magic command, which counts all objects with the `detectors` label.

Note that the various magic commands are only available from the command line, not for use in a bluesky plan function.

In [ ]:
%ct

Compare with the reading when _all_ channels are selected:

In [ ]:
scaler1.select_channels()
%ct

Now, select just the two channels again before continuing:

In [ ]:
scaler1.select_channels(["diode", "I0"])

As noted before, the `%ct` command is only available from the command shell.

### use ophyd to count the scaler

We should learn how to use the underlying Python code to do the same steps.

The first step is to use pure ophyd methods to count and report, then use a bluesky plan to do the same thing.  The ophyd methods are `trigger`, `wait`, and `read`.  The `trigger` and `wait` methods can be chained together:

In [ ]:
scaler1.trigger().wait()

Technically, we should `stage` and `unstage` the object.  **We'll use staging to control the count time of the scaler.**

The ophyd [`.stage()` method](https://nsls-ii.github.io/ophyd/generated/ophyd.device.BlueskyInterface.stage.html?highlight=stage#ophyd.device.BlueskyInterface.stage) prepares the object for its `.trigger()` method, while the `.unstage()` method returns the object's settings to the previous state before the `.stage()` method was called.

In [ ]:
scaler1.stage()
scaler1.trigger().wait()
scaler1.unstage()

Let's find out what happens when `scaler1` is staged.  That's controlled by the contents of a Python dictionary `.stage_sigs`:

In [ ]:
scaler1.stage_sigs

It's empty, so nothing has been preconfigured for us.  Let's make sure that we get to pick the *counting time* (the time to accumulate pulses in the various channels), say 2.0 seconds, when we count here.

In [ ]:
scaler1.stage_sigs["preset_time"] = 2
scaler1.stage_sigs

Show the counting time *before* we count, then `stage`, `trigger`, `wait`, `read`, `unstage`, then finally show the counting time  *after* we count:

In [ ]:
print(f"Scaler configured to count for {scaler1.preset_time.get()}s")
scaler1.stage()
scaler1.trigger().wait()
print(scaler1.read())
scaler1.unstage()
print(f"Scaler configured to count for {scaler1.preset_time.get()}s")

The report from `.read()` includes both values and timestamps (in seconds since the Python [time](https://docs.python.org/3/library/time.html) epoch, UTC).  The structure is a Python dictionary.  This is the low-level method used to collect readings from any ophyd device.  We had to `print()` this since the return result from a command within a sequence is not returned at the end of the sequence, just the return result of the *final* command in the sequence.

See that the scaler counted for 2.1 seconds (a small bug in the scaler simulator it seems, always adds .1 to the count time!).  But before staging, the scaler was configured for 1.0 seconds, and after unstaging, the scaler returned to that value.

**That's how to control the counting time *for a scaler*.**  (Area detectors use different terms.  More on that later.)

<details>
    <summary>about <tt>scaler1_time</tt></summary>

     Of note is the key `scaler1_time` which is the name of the ophyd symbol `scaler1.time` as returned by `scaler1.time.name`:

        In [21]: scaler1.time.name
        Out [21]: 'scaler1_time'

</details>

### use bluesky (the package) to count the scaler

Now, use the bluesky [RunEngine](https://blueskyproject.io/bluesky/generated/bluesky.run_engine.RunEngine.html?highlight=runengine#bluesky.run_engine.RunEngine) (`RE`) to count `scaler1`.  We'll use the bluesky plan ([`bp`](https://blueskyproject.io/bluesky/plans.html?highlight=count#pre-assembled-plans)) called [`count()`](https://blueskyproject.io/bluesky/generated/bluesky.plans.count.html#bluesky.plans.count).  To be consistent with the result returned from `%ct`, we'll include the `noisy` detector.

In [ ]:
RE(bp.count([scaler1,noisy]))

There are many ways to view data from bluesky runs.  We'll pick one simple way, as a [dask](https://dask.org/) table since it is easy to display such structured content in a Jupyter notebook.

In [ ]:
db.v2[-1].primary.to_dask()

As a last action in this section, use the [`listruns()`](https://apstools.readthedocs.io/en/latest/source/_utils.html?highlight=listobjects#apstools.utils.listruns) command from *apstools* to show the (default: 20) most recent runs in the database.  The table shows a short version of the run's unique identifier (`short_uid`), and other more obvious columns of information, truncated to avoid lengthy output.  The name of the [databroker *catalog*](https://blueskyproject.io/databroker/index.html) (`class_2021_03`) is shown before the table.

In [ ]:
listruns()

## temperature _v_ time

work-in-progress

In [ ]:
temperature.position

In [ ]:
temperature.get()

In [ ]:
temperature.readback.get()

In [ ]:
temperature.position

In [ ]:
temperature.read()

In [ ]:
listdevice(temperature)

In [ ]:
RE(bp.count([scaler1,noisy,temperature]))

In [ ]:
RE(bp.count([scaler1,noisy,temperature], num=5))

In [ ]:
RE(bp.count([temperature], num=5))

In [ ]:
RE(bp.count([temperature], num=5, delay=2))

In [ ]:
RE(bp.scan([noisy], temperature, 25, 35, 11))

In [ ]:
%mov temperature 25

In [ ]:
temperature.position

In [ ]:
%movr temperature 1

In [ ]:
RE(bps.mv(temperature, 25))

In [ ]:
RE(bps.mvr(temperature, 1))

## Find a peak: detector _v_ motor

work in progress

## Record an Image

work in progress

----

---

[`%mov`](https://blueskyproject.io/bluesky/magics.html?highlight=label#moving-a-motor) (Magic command) : move a positioner